# 🧑🏽‍🍳 How to Train a Computer Vision Model with Focoos

## 🐍 Setup Focoos

In [ ]:
%pip install 'focoos @ git+https://github.com/FocoosAI/focoos.git'

# 🎨 Fine-tune a model in few steps

This section covers the steps to create a model and train it using the focoos library. The following example demonstrates how to interact with the Focoos API to manage models, datasets, and training jobs.

In this guide, we will perform the following steps:


0. ☁️ [Optional] Connect with Focoos Hub
1. 🎯 Select Pretrained Model
2. 📦 Load a dataset
3. 🏃‍♂️ Train the model
4. 🧪 Test your model
5. 📤 Export your model


## ☁️ [Optional] Connect with FocoosHUB

Focoos can be used without having an accont on the [Focoos Hub](app.focoos.ai). With it, you will unlock additional functionalities, as we will see below. If you have it, just connect to the HUB.

In [ ]:
import os

from focoos import FocoosHUB

FOCOOS_API_KEY = os.getenv(
    "FOCOOS_API_KEY"
)  # write here your API key os set env variable FOCOOS_API_KEY, will be used as default
hub = FocoosHUB(api_key=FOCOOS_API_KEY)

## 🎯 List Pretrained Focoos Models with ModelRegistry

In [ ]:
from focoos import ModelRegistry

model_registry = ModelRegistry()

for m in model_registry.list_models():
    model_info = model_registry.get_model_info(m)
    model_info.pprint()

## Load Pretrained Model with ModelManager

In [ ]:
from focoos import ModelManager

model_name = "fai-detr-l-obj365"
model = ModelManager.get(model_name)
model.model_info.pprint()

## 📦 Download datasets


### Public toy datasets

In [ ]:
from focoos import DATASETS_DIR, DatasetLayout, Task
from focoos.hub.api_client import ApiClient

ds_task = Task.DETECTION


def get_dataset(task: Task):
    if task == Task.SEMSEG:
        ds_name = "balloons-coco-sem.zip"
        layout = DatasetLayout.ROBOFLOW_SEG
    elif task == Task.DETECTION:
        ds_name = "chess-coco-detection.zip"
        layout = DatasetLayout.ROBOFLOW_COCO
    elif task == Task.INSTANCE_SEGMENTATION:
        ds_name = "fire-coco-instseg.zip"
        layout = DatasetLayout.ROBOFLOW_COCO
    else:
        raise ValueError(f"Error: task {task} not supported")
    url = f"https://public.focoos.ai/datasets/{ds_name}"
    api_client = ApiClient()
    api_client.download_ext_file(url, DATASETS_DIR, skip_if_exists=True)
    return ds_name, layout


# Downlaod sample dataset
ds_name, ds_layout = get_dataset(ds_task)

### [Optional] Datasets from focoos Hub

If you want to download a dataset from the hub, you can use the hub to directly store it in your local environment.
Check the reference of your dataset on the platform and use it in the following cell.
In the next cell, we will download a dataset by reference

In [ ]:
hub_datasets = hub.list_remote_datasets()
for dataset in hub_datasets:
    print(dataset.ref, dataset.name, dataset.layout, dataset.task)


ref = None  # place here the ref of the dataset you want to download
if ref is not None:
    dataset = hub.get_remote_dataset(ref)
    dataset_path = dataset.download_data()
    ds_name = dataset_path
    ds_layout = dataset.layout
    ds_task = dataset.task

## AutoDataset and Augmentation
Now that we downloaded the dataset, we can magically 🪄 instanciate the dataset using the `AutoDataset` as will be used in the training. You can optionally specify aumgentations for the training using the `DatasetAugmentation` dataclass.

In [ ]:
from focoos import DatasetSplitType
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import DatasetAugmentations

auto_dataset = AutoDataset(dataset_name=ds_name, task=ds_task, layout=ds_layout)

train_augs = DatasetAugmentations(
    resolution=512,
    color_augmentation=1.0,
    horizontal_flip=0.5,
    vertical_flip=0.0,
    rotation=0.0,
    aspect_ratio=0.0,
    scale_ratio=0.0,
    crop=True,
)
valid_augs = DatasetAugmentations(resolution=512)
# Optionally, you can also get the default augmentations for the task
# train_augs, valid_augs = get_default_by_task(task, 512)

train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=valid_augs.get_augmentations(), split=DatasetSplitType.VAL)

### Visualize
Let's also visualize a few augmented inputs!

In [ ]:
display(train_dataset.preview())

## 🏃‍♂️ Train the model
The next step is to train the model. You can train the model by calling the train method. You need to give it the hyperparameters, encapsulated in the `TrainerArgs`, the datasets and see the magic happens.

In [ ]:
from focoos import TrainerArgs

args = TrainerArgs(
    run_name=f"{model.name}_{train_dataset.name}",  # the name of the experiment
    output_dir="./experiments",  # the folder where the model is saved, DEFAULT  ~/FocoosAI/models"
    batch_size=16,  # how many images in each iteration
    max_iters=500,  # how many iterations lasts the training
    eval_period=200,  # period after we eval the model on the validation (in iterations)
    learning_rate=0.0001,  # learning rate
    weight_decay=0.0001,  # regularization strenght (set it properly to avoid under/over fitting)
    sync_to_hub=False,
)  # Use this to sync model info, weights and metrics on the platform

# Let's go!
model.train(
    args, train_dataset, valid_dataset, hub=None
)  # Hub is optional, if not provided and sync_to_hub is True, will be created automatically

## 🧪 Test your model
Let's visualize some prediction!

In [ ]:
import random

from PIL import Image

index = random.randint(0, len(valid_dataset))

print("Ground truth:")
display(valid_dataset.preview(index, use_augmentations=False))

image = Image.open(valid_dataset[index]["file_name"])
outputs = model.infer(image, annotate=True)

print("Prediction:")
display(Image.fromarray(outputs.image))

## 📤 Export Model and optimize inference 

In [ ]:
from focoos import RuntimeType

infer_model = model.export(runtime_type=RuntimeType.TORCHSCRIPT_32)

infer_model.benchmark(iterations=10)
detections = infer_model.infer(image, threshold=0.5)